# 0. GPT From Scratch

이 노트북은 GPT 논문을 기반으로 간단한 클론코딩을 통해 세익스피어풍의 작품을 생성해내는 언어 모델을 구현한 것입니다.

1. dataset:  tiny shakespere dataset 
2. tokenization : character-level 

언어 모델의 학습 방법론과 트랜스포머 구조에 대한 아주 기초적인 이해를 보유하고 있다고 가정하고 작성을 진행하겠습니다.

# 1. Dataset Preperation & Analysis

In [111]:
with open('./datas/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [112]:
# 데이터셋에 대한 여러 가지를 확인해봅니다. 
print("데이터셋의 총 길이:", len(text))
print("데이터셋의 첫 1000자:\n==============================\n" + text[:1000] + "==============================" )

데이터셋의 총 길이: 1115394
데이터셋의 첫 1000자:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for 

# 2. Tokenizing & Encoding Text - chararcter level

- 간단하게 character level로 토큰화 및 인코딩하기로 결정함. 
- 구현이 간편하지만 인코딩된 토큰이 의미를 담아내지 못하므로
- 모델 성능은 상당히 낮을 것으로 예상됨. 

In [113]:
# Dataset의 모든 character 추출 및 등장 빈도만큼 오름차순 정렬
# character-level이므로 vocab은 등장한 모든 문자의 모음이 됩니다. 
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("vocab:", ''.join(chars))
print("vocab size:", vocab_size)

vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [114]:
# 문자 하나 당 번호 하나를 매깁니다. 
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

# 문자열과 벡터를 오가는 인코딩/디코딩 함수를 만들어줍니다. 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print("========== 예시 ==========")
print(encode("hi there!"))
print(decode(encode("What's your name?")))

========== 예시 ==========
[46, 47, 1, 58, 46, 43, 56, 43, 2]
What's your name?


In [115]:
# 전체 데이터셋 인코딩

# 위 인코더로 text를 인코딩한 뒤 torch tensor 형태로 전체 데이터를 준비해둡니다. 
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print("data의 shape:", data.shape)
print("data의 dtype:", data.dtype)
# print(data[:1000])

data의 shape: torch.Size([1115394])
data의 dtype: torch.int64


# 3. Train / Validation / Test Split

In [116]:
# train과 test data를 분리한다.
# 비율은 약 10%로 정함.
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

이 데이터들을 "batch * context length" shape의 tensor 단위로 묶어낸다. 

In [117]:
# 모델의 context window size를 정한다. 
context_length = 8
print(train_data[:context_length + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [118]:
# 예시
x = train_data[:context_length]
y = train_data[1: context_length + 1]
for i in range(context_length):
    context = x[:i + 1]
    target = y[i]
    print(f'input이 {context}일 때의 target: {target}')

input이 tensor([18])일 때의 target: 47
input이 tensor([18, 47])일 때의 target: 56
input이 tensor([18, 47, 56])일 때의 target: 57
input이 tensor([18, 47, 56, 57])일 때의 target: 58
input이 tensor([18, 47, 56, 57, 58])일 때의 target: 1
input이 tensor([18, 47, 56, 57, 58,  1])일 때의 target: 15
input이 tensor([18, 47, 56, 57, 58,  1, 15])일 때의 target: 47
input이 tensor([18, 47, 56, 57, 58,  1, 15, 47])일 때의 target: 58


In [119]:
# 모델의 batch size를 정한다. 
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    '''
    get_batch: 

    split: 어떤 데이터에 대한 batch를 불러올 건지 결정. 
    'train'인 경우 train_data에 대해서, 아닌 경우 val_data에 대해서 가져옴.
    '''
    # 데이터 불러오기
    data = train_data if split == 'train' else val_data 

    # batch size 개수의 랜덤한 offset 생성. 
    ix = torch.randint(len(data) - context_length, (batch_size,)) 

    # 랜덤 생성한 시작지점 ix에 대한 x, y 가져오기
    x = torch.stack([data[i : i + context_length] for i in ix])
    y = torch.stack([data[i + 1 : i + context_length + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('입력: ')
print(xb.shape)
print(xb)

for l in xb.tolist():
    print(decode(l))
print("========================================")
print('target: ')
print(yb.shape)
print(yb)

for l in yb.tolist():
    print(decode(l))

print('========================================')

for btch in range(batch_size):
    for ctxt in range(context_length):
        context = xb[btch, : ctxt + 1]
        target = yb[btch, ctxt]
        print(f'입력이 {context.tolist()}일 때 target: {target}')

입력: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Let's he
for that
nt that 
MEO:
I p
target: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
et's hea
or that 
t that h
EO:
I pa
입력이 [24]일 때 target: 43
입력이 [24, 43]일 때 target: 58
입력이 [24, 43, 58]일 때 target: 5
입력이 [24, 43, 58, 5]일 때 target: 57
입력이 [24, 43, 58, 5, 57]일 때 target: 1
입력이 [24, 43, 58, 5, 57, 1]일 때 target: 46
입력이 [24, 43, 58, 5, 57, 1, 46]일 때 target: 43
입력이 [24, 43, 58, 5, 57, 1, 46, 43]일 때 target: 39
입력이 [44]일 때 target: 53
입력이 [44, 53]일 때 target: 56
입력이 [44, 53, 56]일 때 target: 1
입력이 [44, 53, 56, 1]일 때 target: 58
입력이 [44, 53, 56, 1, 58]일 때 target: 46
입력이 [44, 53, 56, 1, 58, 46]일 때 target: 39
입력이 [44, 53, 56, 1, 58, 46, 39]일 때 target: 58
입력이 [44, 53,

- get_batch 함수를 통해 context_length 길이의 '문장'을(정확히 말하면 문장은 아님) batch_size 개만큼 가져옴.
- 즉, batch_size x context_length shape의 tensor x, y를 구성.
- x는 주어진 context, y는 주어진 context에 대한 정답(target)

In [120]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


# 4. Model 정의

### 1. Bigram Language Model

In [121]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModelVer1(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # 토큰 임베딩 테이블 정의
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):

        '''
        idx: (B, T) tensor. x가 들어갈 위치(input)
        targets: (B, T) tensor. y가 들어갈 위치(target)
        logits: (B, T, C) tensor. 여기서 C는 Channel로, 여기서는 vocab_size를 나타냄. 

        즉, C개의 문자가 각 (B, T)의 예측 target에 해당할 각각의 logit을 나타낸 것. 
        '''
        
        # 토큰 임베딩 테이블을 lookup하여 logit 구함
        logits = self.token_embedding_table(idx)

        # 구한 logit과 target을 비교하여 loss 계산

        # 차원을 맞추기 위한 작업
        # loss를 계산하려면 B, C, T의 shape이어야 함. 
        # logit과 target을 모두 B*T로 차원 축소해서 길게 늘린 뒤 비교
        # 자세한 건 pytorch 공식문서 읽어보기!!!! 
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B*T) # -1
        loss = F.cross_entropy(logits, targets) # Negative Log Likelihood 사용 
        # -ln(1/65) ~= 4.17

        return logits, loss
    
m = BigramLanguageModelVer1(vocab_size)
logits, loss = m(xb, yb) # forward 함수 실행됨
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


이 모델은 각 토큰끼리의 정보 교환(어텐션, 피드 포워딩 등)을 일절 하지 않은 채 '현재 해당하는 토큰 임베딩값'만으로 다음 토큰을 예측하는 모델임.

랜덤 초기화된 임베딩 테이블을 그대로 사용해 예측을 수행했기 때문에 단순 찍기에 달하는 성능을 보인다... (당연한 게 이 모델이 단순 찍기 모델이기 때문이다)

찍기의 loss 기댓값은 -ln(1/65) ~= 4.17이지만 우리의 모델은 4.87을 보이는 중. 

이제 이 모델로 생성을 해보자!

### BigramLanguageModel - generate 추가 버전

In [122]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # 토큰 임베딩 테이블 정의
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # target 여부를 optional로 두어야!
    def forward(self, idx, targets=None):

        '''
        idx: (B, T) tensor. x가 들어갈 위치(input)
        targets: (B, T) tensor. y가 들어갈 위치(target)
        logits: (B, T, C) tensor. 여기서 C는 Channel로, 여기서는 vocab_size를 나타냄. 

        즉, C개의 문자가 각 (B, T)의 예측 target에 해당할 각각의 logit을 나타낸 것. 
        '''
        
        # 토큰 임베딩 테이블을 lookup하여 logit 구함
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            # 구한 logit과 target을 비교하여 loss 계산

            # 차원을 맞추기 위한 작업
            # loss를 계산하려면 B, C, T의 shape이어야 함. 
            # logit과 target을 모두 B*T로 차원 축소해서 길게 늘린 뒤 비교
            # 자세한 건 pytorch 공식문서 읽어보기!!!! 
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B*T) # -1
            loss = F.cross_entropy(logits, targets) # Negative Log Likelihood 사용 
            # -ln(1/65) ~= 4.17

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        '''
        idx: x
        max_new_tokens: 생성 가능한 최대 토큰 개수수
        '''

        for _ in range(max_new_tokens):
            
            # forward로 logit과 loss 계산
            logits, loss = self(idx)

            # 각 batch의 마지막 문자에 대한 예측만 취함(Bi-gram이기 때문!)
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)

            # 여기서 들 수 있는 의문점: 어차피 마지막 문자에 대한 예측만 볼 거면서 왜 
            # 계속 늘어나는 idx를 그대로 forward에 넣고 있을까?
            # 나중에 모델을 N-gram으로 확장할 때 함수의 형태를 최대한 일관되게 하려고 함
            # 연산은 비효율적이지만 공부하는데에는 이렇게 하는게 좋을 듯. 

            # Softmax 거쳐서 확률 계산
            probs = F.softmax(logits, dim = -1) # (B, C)

            # 분포에 따라 추출(예측 생성성)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # 각 batch에 대한 예측 결과를 x의 맨 끝에 append
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
        
        # 위 loop가 돌면서 idx는 (B, T + 1), (B, T + 2), ... , (B, T + max_new_tokens)의 shape이 됨.
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # forward 함수 실행됨
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [123]:
idx = torch.zeros((1, 1), dtype=torch.long) # 0에서 생성 시작. 
# 0은 \n 문자임!
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


역시 개판임을 볼 수 있다!
이제 이 모델을 학습시켜 보자!


In [124]:
# optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [125]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [126]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenou


오... 이제 좀 뭔가 따라하려는 흔적이 보이기 시작한다!

여기까지 내용을 bigram.py로 저장

### 2. Self-Attention Model
본격적으로 들어가기 전에...
##### 1. Mathematical Background for Self-Attention

- 이전 토큰들과 상호작용하는 가장 쉬운 방법은? sum 또는 mean!
- 어텐션만큼 강한 메커니즘은 아니고, 정보의 손실도 크지만 일단 이걸로 진행해보자!

In [127]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [128]:
# version 1 : Double Loop로 평균 계산산

xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):

        # xprev : (t, C)짜리 현재 시점 이전(현재 포함)의 모든 latent 벡터 모음
        xprev = x[b, :t + 1]

        # 모든 latent vector 평균내기
        # bow는 bag-of-words의 줄임말이다. 그 bag of words 맞다. 
        xbow[b, t] = torch.mean(xprev, 0)

In [129]:
print(x[0])
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


하지만 이렇게 이중루프를 쓰면 연산 효율이 너무 떨어진다. 행렬연산을 쓰자. 어떻게 해야 할까?
- torch.tril : Lower Triangular Matrix로 만들어줌.
- 이를 ones 행렬과 함께 써서 곱하면 현재 시점까지의 누적합이 된다!
- normalize까지 하면 누적 평균을 구할수도 있다!

In [130]:
# Toy Example
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
--


In [137]:
# version 2: matrix multiplication 사용

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) --> (B, T, C)
# (T, T) -> (B, T, T)로 확장 후 연산산
print(xbow[0])
print('===')
print(xbow2[0])
torch.allclose(xbow, xbow2)

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
===
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


False

In [138]:
# version 3: Softmax 사용 

# lower triangluar ONES matrix 생성
tril = torch.tril(torch.ones(T, T))

# 0으로 초기화
wei = torch.zeros((T, T))

# tril의 형태에 따라 lower triangular zero matrix 생성, 위는 -INF로 채움. 
wei = wei.masked_fill(tril == 0, float('-inf'))

# 이를 Softmax에 통과시키면 위에서 구했던 weight matrix와 동일한 결과. 
wei = F.softmax(wei, dim=-1)

xbow3 = wei @ x # (T, T) @ (B, T, C) --> (B, T, C)
# (T, T) -> (B, T, T)로 확장 후 연산

print(xbow[0])
print('===')
print(xbow3[0])
torch.allclose(xbow, xbow3)

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
===
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


False

여기까지의 내용을 v2.py로 저장